## INPUT PIPELINE

Assigning proper tag to a particular question is crucial for the model to perform well. Since each sample has tags relating to both the data structures and algorithms used to 
solve those samples, it is necessary to separate this overlap. <br>

It is also essential that no sample should have more than one target tag. So whenever, targets are created it is absolutely essential to verify that there is no overlap of target tags.<br>

This notebook is used as the **input pipeline** for the question tagging notebook. Any variation of the original dataset with various target tags to be fed into the model, will be prepared here

In [1]:
import pandas as pd
import numpy as np
from collections import defaultdict

### Data Structures

In [2]:
df = pd.read_csv("../datasets/question_tagging_dataset.csv")

In [3]:
data_structures = {
    "array", "string", "graph"
}

In [4]:
pair_freqs = defaultdict(int)
for row in range(df.shape[0]):
    tags = list(df.iloc[row, 1:8].values)
    for i in range(len(tags)):
        if tags[i] == "No tag" or tags[i] not in data_structures:
            break
        for j in range(i+1, len(tags)):
            if tags[j] == "No tag" or tags[j] not in data_structures:
                break
            if tags[i] in data_structures and tags[j] in data_structures:
                pair_freqs[tuple(sorted((tags[i], tags[j])))] += 1

pair_freqs

defaultdict(int,
            {('array', 'array'): 48,
             ('array', 'string'): 57,
             ('array', 'graph'): 106,
             ('graph', 'graph'): 82,
             ('graph', 'string'): 7,
             ('string', 'string'): 6})

In [5]:
drop_inds = []
qns = df[["stop_words_removed_qns", "tag1"]]
for i in range(df.shape[0]):
    tags = set(df.iloc[i, 1:8].values)
    if len(tags.intersection(set(data_structures))) == 1:
        qns.iat[i, 1] = list(data_structures.intersection(tags))[0]
    else:
        drop_inds.append(i)

print(len(drop_inds))

7049


In [6]:
qns.drop(drop_inds, axis=0, inplace=True)
qns.reset_index(drop=True, inplace=True)

/tmp/ipykernel_6028/1037782737.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  qns.drop(drop_inds, axis=0, inplace=True)


In [7]:
qns["tag1"].value_counts()

graph     1225
array      944
string     843
Name: tag1, dtype: int64

In [8]:
qns.to_csv("../datasets/array_graph_string.csv", index=False)

### Algorithms

In [9]:
algorithms = {"brute force", "greedy", "dynamic programming"}

In [10]:
pair_freqs = defaultdict(int)
for row in range(df.shape[0]):
    tags = list(df.iloc[row, 1:8].values)
    
    for i in range(len(tags)):
        if tags[i] == "No tag" or tags[i] not in algorithms:
            break
        
        for j in range(i+1, len(tags)):
            if tags[j] == "No tag":
                break
            if tags[j] not in algorithms:
                continue
            elif tags[i] in algorithms and tags[j] in algorithms:
                pair_freqs[tuple(sorted((tags[i], tags[j])))] += 1

In [11]:
for k in pair_freqs:
    print(f"{k}: {pair_freqs[k]}")

('brute force', 'dynamic programming'): 162
('brute force', 'greedy'): 405
('dynamic programming', 'greedy'): 325


In [12]:
drop_inds = []
qns = df[["stop_words_removed_qns", "tag1"]]
for i in range(df.shape[0]):
    tags = set(df.iloc[i, 1:8].values)
    if len(tags.intersection(set(algorithms))) == 1:
        qns.iat[i, 1] = list(tags.intersection(set(algorithms)))[0]
    else:
        drop_inds.append(i)
len(drop_inds)

5405

In [13]:
qns.drop(drop_inds, inplace=True)
qns.reset_index(inplace=True, drop=True)

/tmp/ipykernel_6028/2664307388.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  qns.drop(drop_inds, inplace=True)


In [14]:
qns.head(3)

,stop_words_removed_qns,tag1
0,for birthday Kevin receive set pairwise distin...,greedy
1,you work quality control department technical ...,greedy
2,little Misha go programming club solve it stra...,brute force


In [15]:
qns.to_csv("../datasets/greedy_brute_force_dp.csv", index=False)